# Attention mechanizmus a transformery

Na minulom cvičení sme sa pozreli na to, ako dokážeme využiť rekurentné neurónové siete na spracovanie prirodzeného jazyka v kontexte analýzy sentimentu, a generovania textu. Tiež sme zadefinovali rôzne typy úloh so sekvenčnými dátami, okrem iného aj generovanie many-to-many, kde vstupom je sekvencia údajov a na výstupe sieť tiež vyprodukuje sekvenciu, práve preto sa takýto prípad použitia nazýva aj **sequence-to-sequence** (**seq2seq**). Špeciálnym prípadom seq2seq úloh bol prípad, kde sme výstupnú sekvenciu generovali so skĺzom, čo znamenalo, že sieť musela najprv spracovať všetky vstupné údaje, až neskôr dokázala vygenerovať výstup. Nevýhodou takéhoto prístupu je, že posledný skrytý stav, na základe ktorého sa vygeneruje výstup, musí v sebe obsahovať všetky relevantné informácie z celého vstupu.

Na dnešnom cvičení sa pozrieme na to, ako dokáže využitie attention mechanizmu pomôcť pri riešení tohto problému. Následne sa budeme zaoberať self-attention prístupom, a dostaneme sa k transformerom a veľkým jazykovým modelom.

## Attention mechanizmus v RNN

Attention mechanizmus umožňuje rekurentnej sieti pristupovať k všetkým vstupným dátam pri každom kroku generovania výstupu. Takýto prístup však môže byť priveľký, a práve preto zavedieme nové typy váh, tzv. *attention váhy*, ktoré vyjadrujú dôležitosť daného vstupu pre generovanie výstupu v danom časovom okamihu.

Formálne môžeme povedať, že pre danú vstupnú sekvenciu $(x^{(1)}, x^{(2)}, ... x^{(T)})$ attention mechanizmus priradí každému prvku $x^{(i)}$, resp. jeho skrytej reprezentácii, váhu, a tak umožní modelu rozhodnúť sa, na ktorú časť sekvencie sa má sústrediť.

Nasledujúci obrázok znázorňuje základný princíp integrovania attention mechanizmu do rekurentnej siete:

![](lab09/attention-rnn.jpg)

### Obojsmerná rekurentná sieť

Prvým komponentom riešenia je obojsmerná rekurentná sieť, ktorá generuje tzv. *context vector* $c_{i}$, čo predstavuje rozšírenú verziu vstupného vektora $x^{(i)}$ o informácie zo všetkých vstupných prvkov. Prvá sieť je obojsmerná z toho dôvodu, že vstupné údaje spracuje aj v pôvodnom poradí $(1...T)$ ako aj $(T...1)$. Motiváciou takéhoto prístupu je zámer brať do úvahy možný vplyv nasledujúcich prvkov vstupu na aktuálny vstup.

Vzhľadom na dvojsmerné spracovanie vstupu, neurónová sieť vygeneruje dva skryté stavy pre každý vstup: $h_{F}^{(i)}$ pre dopredný prechod a $h_{B}^{(i)}$ pre spätný prechod. Tieto dva stavy sa spoja konkatenáciou a tak vznikne celkový skrytý stav $h^{(i)}$.

### Generovanie výstupu

Druhá neurónová sieť, ktorá generuje výstup, okrem skrytých stavov dostane aj *context vector*, ktorý je vypočítaný nasledovne:

$$c_{i} = \sum_{j=1}^{T} \alpha_{ij} h^{(i)},$$

kde $\alpha_{ij}$ predstavuje attention váhu pre vstupnú postupnosť $j = (1...T)$ pre *i*-tý vstupný prvok. To znamená, že každý vstupný údaj má pridelenú vlastnú sadu attention váh.

Druhá rekurentná neurónová sieť potom generuje výstup na základe vstupných informácií pomocou vlastných skrytých stavov $s^{(i)}$, ktoré závisia od predošlého skrytého stavu $s^{(i - 1)}$, od predošlého očakávaného výstupného slova $y^{(i - 1)}$ a od *context vector*a $c^{(i)}$. Očakávaný výstup je samozrejme dostupný iba počas trénovania, počas inferencie sa použije predikovaný výstup z predošlého kroku, t.j. $o^{(i - 1)}$.

### Výpočet attention váh

Každá attention váha je definovaná pre vstup (anotácie zo skrytých stavov) a výstup (kontext), a práve preto používame dva indexy (*j* pre vstup, *i* pre výstup). Váhu vypočítame normalizáciou *alignment skóre* $e_{ij}$, ktorá vyjadruje, ako veľmi je vstup okolo pozície *j* v súlade s výstupom na pozícii *i*.

$$\alpha _{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T} \exp(e_{ik})}$$

Normalizácia prebieha rovnako ako pri softmax aktivácii, to znamená, že attention váhy pre výstup *i* budú mať súčet 1.

## Self-attention mechanizmus

Transformery na rozdiel od predošlej rekurentnej siete používajú self-attention mechanizmus, ktorý nevyžaduje rekurentný proces, teda transformer dokáže spracovať vstupnú sekvenciu v jednom kroku a nemusí na to využiť iteráciu. Ďalším zásadným rozdielom je, že kým attention mechanizmus akokeby prepájal dve siete (enkóder a dekóder), tak sa self-attention zaoberá iba so vstupmi a neprepája ich na iný modul.

### Self-attention bez učenia

Cieľom self-attention mechanizmu je namodelovať súvislosti medzi jednotlivými vstupnými vektormi $x^{(1)}, x^{(2)}, ... x^{(T)}$, ktoré bude reprezentovať vo výstupných vektoroch $z^{(1)}, z^{(2)}, ... z^{(T)}$ (označenie $o^{(i)}$ ponecháme pre celkový výstup siete). Výpočet pozostáva z troch fáz:

1. výpočet váh dôležitosti na základe podobnosti všetkých prvkov vstupnej sekvencie k aktuálnemu prvku (vstup bol namapovaný do číselnej podoby, a prešiel embeddingom)

$$\omega _{ij} = {x^{(i)}}^{T} x^{(j)}$$

2. normalizácia váh (zvyčajne pomocou softmax)

$$\alpha _{ij} = \frac{\exp(\omega_{ij})}{\sum_{j=1}^{T} \exp(\omega_{ij})}$$

3. výpočet hodnoty attention

$$z^{(i)} = \sum_{j=1}^{T} \alpha_{ij} x^{(j)}$$

In [ ]:
import torch

sentence = torch.tensor(
    [0,  # please
     7,  # help
     1,  # me
     2,  # translate
     5,  # this
     6]  # sentence
)
print(sentence)

In [ ]:
torch.manual_seed(123)
embed = torch.nn.Embedding(10, 16)
embedded_sentence = embed(sentence).detach()
print(embedded_sentence)

In [ ]:
# 1. krok - výpočet váh dôležitosti
omega_mat = embedded_sentence.matmul(embedded_sentence.T)
print(omega_mat)

In [ ]:
# 2. krok - normalizácia
import torch.nn.functional as F

attention_weights = F.softmax(omega_mat, dim=1)
print(attention_weights)
print(attention_weights.sum(axis=1))

In [ ]:
# 3. krok - výpočet kontext vektorov
context_vectors = torch.matmul(attention_weights, embedded_sentence)
print(context_vectors)

### Self-attention s učením

Existuje niekoľko foriem attention mechanizmu, jedna z najvýznamnejších je **scaled dot-product attention** použitý aj v transformeroch. Hore uvedený attention mechanizmus rozširuje o tri matice váh, ktoré sú trénovateľné:

* query sequence: $q^{(i)} = U_{q} x^{(i)}$
* key sequence: $k^{(i)} = U_{k} x^{(i)}$
* value sequence: $v^{(i)} = U_{v} x^{(i)}$.

Tieto matice zvyčajne majú rozmery totožné s embeddingom, teda $d_q = d_k = d_v = d$, kde *d* je dĺžka vektoru jedného vstupu. V tejto forme attention mechanizmus výpočet výstupu prebieha nasledovne:

1. výpočet váh dôležitosti pomocou query a key postupností

$$\omega _{ij} = {q^{(i)}}^{T} k^{(j)}$$

2. normalizácia váh (zvyčajne pomocou softmax) na približne rovnakú dĺžku (zvyčajne $m = d_k$)

$$\alpha_{ij} = softmax (\frac{\omega_{ij}}{\sqrt{m}})$$

3. výpočet hodnoty attention

$$z^{(i)} = \sum_{j=1}^{T} \alpha_{ij} v^{(j)}$$

In [ ]:
torch.manual_seed(123)
d = embedded_sentence.shape[1]
U_q = torch.rand(d, d)
U_k = torch.rand(d, d)
U_v = torch.rand(d, d)

queries = U_q.matmul(embedded_sentence.T).T
keys = U_k.matmul(embedded_sentence.T).T
values = U_v.matmul(embedded_sentence.T).T

In [ ]:
# 1. krok - výpočet váh dôležitosti
omega_2 = queries[1].matmul(keys.T)
print(omega_2)

In [ ]:
# 2. krok - normalizácia
attention_weights_2 = F.softmax(omega_2 / d**0.5, dim=0)
print(attention_weights_2)

In [ ]:
# 3. krok - výpočet kontext vektorov
context_vector_2 = attention_weights_2.matmul(values)
print(context_vector_2)

## Transformery

Transformery boli predstavené v roku 2017 na konferencii NeurIPS. Sú síce založené na koncepte attentionu pre rekurentné siete, avšak fungujú omnoho efektívnejšie aj vďaka tomu, že vynechajú rekurentnú časť. Základná architektúra transformerov je znázornená nižšie:

![](lab09/transformer.jpg)

### Enkóder

Úlohou prvej časti transformera, teda enkódera, je namapovať vstupnú sekvenciu $X = (x^{(1)}, x^{(2)}, ... x^{(T)})$ do spojitej reprezentácie $Z = (z^{(1)}, z^{(2)}, ... z^{(T)})$. V pôvodnom článku sa enkóder skladal zo šiestich identických vrstiev, ich počet je však ďalším hyperparametrom.

Dôležitým rozdielom oproti predošlým mechanizmom je použitie **multi-head self-attention**, čo je v podstate niekoľkonásobná paralelná aplikácia **scaled dot-product attention** mechanizmu, akokeby sme použili viac konvolučných filtrov v jednej vrstve. Jeden *head* komponent sa skladá z troch matíc (queries, keys a values), pri multi-head self-attention teda vygenerujeme niekoľko takýchto matíc, a pre každú z nich vypočítame výstup podobným spôsobom, ako sme to robili pred tým. Rovnako ako pri konvolúcii, jednotlivé *head* komponenty sa dokážu sústrediť na rôzne časti vstupných údajov.

Následne tieto vektory spojíme konkatenáciou do tvaru $d_v \times h$, kde *h* je počet head komponentov. Tento rozšírený vektor upravíme do pôvodného tvaru cez plne prepojenú vrstvu.

### Dekóder

Dekóder pracuje rovnako s niekoľkými identickými blokmi, avšak v jednom bloku použijeme ďalší typ attention mechanizmu, a to **masked multi-head attention**. Maskovanie dosiahneme napríklad prekrytím slov, ktoré nasledujú po aktuálnom slove, čím sa snažíme nasimulovať fungovanie modelu počas inferencie. Takéto maskovanie však aplikujeme iba na postupnosť výstupov, vstupné údaje naďalej sú spracované obojsmerne cez enkóder.

Vstupom do dekóder bloku sú výstup enkódera, ako aj zakódovaný výstup z predošlého kroku, ktorý prechádza práve masked attention mechanizmom. Po ďalšom prechode multi-head attention mechanizmom plne prepojená vrstva vygeneruje výstup dekódera, teda pravdepodobnosti pridelené jednotlivým výstupným slovám. Vybrané potom bude slovo s najväčšou predikovanou pravdepodobnosťou.

### Pozičné enkódovanie

Dôležitou súčasťou transformerov je použitie pozičného enkódovania, ktoré slúži na zachytávanie informácií o poradí slov vo vstupe. Tento prístup je potrebný z toho dôvodu, že attention mechanizmy sú invariantné voči permutáciám, čo by znamenalo, že by transformery ignorovali poradie slov, čo môže spôsobiť problémy, napr.:

1. Mary gave Bertie a flower.
2. Bertie gave Mary a flower.

Pôvodné transformery použili sínusoidálne enkódovanie, ktoré umožní, aby rovnaké slová na rôznych pozíciach mali trošku odlišné enkódovanie cez pripočítanie vektoru k vstupnému vektoru:

$$PE_{(i, 2k)} = \sin (pos / 10000^{2k/d_{model}}),$$

kde *i* je pozícia slova a *k* je dĺžka embeddingu vstupných slov (aby sme vedeli sčítať vektory). Okrem uvedeného absolútneho enkódingu môžeme použiť aj relatívne, ktoré zohľadňuje iba relatívnu pozíciu slov, a tým pádom je invariantné k posunom viet v celkovom texte.

Okrem pozičného enkódovania, transformery používajú aj normalizáciu vrstvy (*layer normalization*), čo umožňuje efektívnejšiu paralelizáciu v porovnaní so štandardným BatchNormom.

## Veľké jazykové modely

Kľúčovú úlohu pri úspechu veľkých jazykových modelov založených na transformeroch zohrával **self-supervised learning**, alebo **unsupervised pre-training**, ktorý využíva špecifické vlastnosti textových údajov pri trénovaní. To znamená, že nepotrebujeme veľké množstvo olabelovaných dát, namiesto toho model natrénujeme napríklad na predikciu ďalšieho slova (vždy prístupné pri textoch), a neskôr doladíme na konkrétny príklad použitia s menším množstvom dát.

Použitie jazykových modelov zvyčajne nasleduje dva prístupy: prístup založený na príznakoch, alebo doladenie. Príkladom prístupu založeného na príznakoch je model **ELMo**, ktorý slúži na embedding a bol trénovaný náhodným maskovaním 15 percent slov (teda neboli zakryté nasledujúce slová, ale náhodné). Pri doladení priamo aktualizujeme váhy predtrénovaného modelu a topológiu zvyčajne rozšírime o minimálne jednu (výstupnú) vrstvu.

Najpoužívanejšie jazkové modely sú GPT, BERT a BART.

### GPT

GPT (**Generative Pre-trained Transformer**) je séria jazykových modelov primárne určených pre generovanie textu. GPT-1 (110M parametrov), ktorý vyšiel v roku 2018, sa skladá iba z dekódera bez enkóderovej časti, a z ďalšej vrstvy, ktorá sa používa počas ladenia modelu. GPT-1 spracuje text iba v jednom smere, to znamená, že pri predikcii nasledujúceho slova sa spolieha iba na predchádzajúce slová, generovanie prebieha po jednotlivých slovách. Počas trénovania sa stále použijú správne slová, počas inferencie do ďalšieho kroku je pridané modelom vygenerované slovo.

Silnou stránkou GPT-1 modelu bola výkonnosť na **zero-shot** úlohách, teda na úlohách, z ktorých nevidel trénovacie príklady. Práve táto adaptabilita inšpirovala vývojárov na ďalšiu generalizáciu modelu vo verzii GPT-2 (1,5B parametrov) cez pridanie kontextu k vstupu, ako napríklad inštrukcia *translate to Slovak* pri strojovom preklade.

Verzia GPT-3 (175B parametrov) sa nesústreďovala na zero-shot, ale na **few-shot** úlohy, t.j. model má k dispozícii malý počet príkladov, z ktorých musí zovšeobecniť rýchlo svoje správanie, čo lepšie zodpovedá ľudskému učeniu. Ďalším rozdielom oproti GPT-2 je použitie **sparse attention**, kde sa model sústredí iba na podmnožinu všetkých prvkov s limitovaným počtom.

GPT modely môžu byť použité aj pre klasifikáciu, v tomto prípade vstup musí byť tokenizovaný a postupne zadaný do modelu. Model je rozšírený o spomínanú vrstvu, ktorá na výstupe má jednu (alebo niekoľko) hodnotu. Pre komplikovanejšie prípady použitia, ako je odpovedanie otázok, je potrebné špeciálne predspracovanie údajov.

### BERT

BERT (**Bidirectional Encoder Representations from Transformers** - 345M parametrov), na rozdiel od GPT, obsahuje enkóder využívajúci obojsmerné spracovanie vstupu (resp. spracovanie bez smeru, keďže celý vstup sa spracuje naraz). To znamená, že enkódovanie slova závisí od predošlých aj nasledujúcich slov, čo spôsobuje kvalitnejšie enkódovanie pre úlohy ako klasifikácia.

BERT využíva tri typy embeddingu: už spomínané token a pozičné embeddingy, ako aj segmentový embedding, ktorý identifikuje vety alebo väčšie celky slov. Potreba embeddingu súvisí s trénovaním BERTu, ktoré prebiehalo aj spôsobom *next-sentence prediction*, kde model musí povedať, či dvojica viet nasleduje po sebe alebo nie.

Okrem toho BERT využíval aj maskovanie pri trénovaní, pričom náhodne bolo vybratých 15 percent tokenov na zamaskovanie, z toho 10 percent bolo ponechaných v pôvodnom stave, 10 percent bolo zamenených za iné náhodné slovo, a zvyšok bol zakrytý.

Pri klasifikácii vo vstupe sa použije špeciálny token `[CLS]`, ktorý je potom nahradený modelom podľa výsledku klasifikácie. Typickými prípadmi použitia BERT modelu sú klasifikácia dvojice viet, klasifikácia jednej vety, odpovedanie na otázky a tagovanie jednej vety. Prvé dve vyžadujú minimálne zmeny v architektúre, kým ostatné vyžadujú úpravu výstupu pre tokeny (predikcie sú generované pre každý jeden token a nie pre celú vetu).

### BART

BART (**Bidirectional and Auto-Regressive Transformer**) kombinuje výhody GPT pre generovanie textu a BERT pre klasifikáciu tak, že obsahuje aj obojsmerný enkóder ako aj autoregresívny dekóder, ktorý funguje zľava doprava. Veľkým rozdielom oproti pôvodnému transformeru je to, že BART používa iba vstupné údaje ako vstup, teda nevyžaduje očakávaný výstup (v pôvodnom transformeri text v inom jazyku). Očakávaný výstup sa stále používa pri výpočte chyby, ale nie je priamym vstupom do modelu.

V BART modeli, vstup je najprv skorumpovaný a následne enkódovaný. Enkódovaný vstup je potom dodaný dekóderu spolu s vygenerovanými tokenmi. Rozdiel medzi výstupom z enkódera a pôvodným textom sa použije na trénovanie enkódera. Korupcia vstupu prebieha pomocou rôznych metód: zamaskovanie tokenov, vymazanie tokenov, doplnenie textu, permutácia vety, rotácia dokumentu.

BART môže byť použitý na:
 - klasifikáciu sekvencie - k vstupu pridáme špeciálny token, ktorý je nahradený výsledkom klasifikácie; vstup nie je skorumpovaný.
 - klasifikáciu tokenov - model priamo použije generované reprezentácie na predikciu pre každý token.
 - generovanie sekvencie - prebieha viac ako sumarizácia, kde je najprv spracovaný celý vstup a následne sa vygeneruje reakcia.
 - strojový preklad - celý BART model je použitý ako čistý predtrénovaný dekóder, a k nemu je pridaný nový enkóder s náhodnými váhami. Nová štruktúra je potom dotrénovaná najprv s freezenutými váhami dekódera, a následnou aktualizáciou všetkých parametrov.

## Príklady

* [Generovanie textu pomocou GPT-2](https://github.com/rasbt/machine-learning-book/blob/main/ch16/ch16-part2-gpt2.ipynb)
* [Dotrénovanie BERT modelu](https://github.com/rasbt/machine-learning-book/blob/main/ch16/ch16-part3-bert.ipynb)

## Použité zdroje

* Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio. "Neural machine translation by jointly learning to align and translate." arXiv preprint arXiv:1409.0473 (2014).
* Vaswani, Ashish, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. "Attention is all you need." Advances in neural information processing systems 30 (2017).
* Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer normalization." arXiv preprint arXiv:1607.06450 (2016).
* Raschka, Sebastian, Yuxi Hayden Liu, Vahid Mirjalili, and Dmytro Dzhulgakov. Machine Learning with PyTorch and Scikit-Learn: Develop machine learning and deep learning models with Python. Packt Publishing Ltd, 2022. Kapitola 16

**GPT:**
* Radford, Alec, Karthik Narasimhan, Tim Salimans, and Ilya Sutskever. "Improving language understanding by generative pre-training." (2018).
* Radford, Alec, Jeffrey Wu, Rewon Child, David Luan, Dario Amodei, and Ilya Sutskever. "Language models are unsupervised multitask learners." OpenAI blog 1, no. 8 (2019): 9.
* Brown, Tom, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared D. Kaplan, Prafulla Dhariwal, Arvind Neelakantan et al. "Language models are few-shot learners." Advances in neural information processing systems 33 (2020): 1877-1901.
* Child, Rewon, Scott Gray, Alec Radford, and Ilya Sutskever. "Generating long sequences with sparse transformers." arXiv preprint arXiv:1904.10509 (2019).

**BERT:**
* Devlin, Jacob, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).

**BART:**
* Lewis, Mike, Yinhan Liu, Naman Goyal, Marjan Ghazvininejad, Abdelrahman Mohamed, Omer Levy, Ves Stoyanov, and Luke Zettlemoyer. "Bart: Denoising sequence-to-sequence pre-training for natural language generation, translation, and comprehension." arXiv preprint arXiv:1910.13461 (2019).